# BTC Prediction With LSTM

In [45]:
import csv
import numpy as np
from datetime import date, datetime, timedelta
from lstm import Machine
import pandas as pd
import tensorflow as tf
from plotly.offline import init_notebook_mode
import cufflinks as cf

MODEL_PATH = "data/lstm.hd5"
CSV_PATH = "data/binance-BTCUSD.csv"
TRAIN_TEST_RATE = 0.85
NUM_SEQ = 30
DUR_PRED = 15

cf.go_offline()
init_notebook_mode(connected=True)

with open(CSV_PATH) as fin:
    reader = [dict(item) for item in csv.DictReader(fin)]

machine = Machine(reader)
(_, _, X_test, Y_test) = machine.split_data(
    TRAIN_TEST_RATE, NUM_SEQ, DUR_PRED
)

base_date = datetime.utcfromtimestamp(
    int(reader[0]['openTime']) / 1000
).date()
end_date = datetime.utcfromtimestamp(
    int(reader[-1]['openTime']) / 1000
).date()

print(base_date, end_date)

user_date = None

while not user_date:
    try:
        user_date = date.fromisoformat(input(
            "What date you want to see? ("
            f"min: {base_date.isoformat()} "
            f"max: {end_date.isoformat()}) "
        ))
        if not (base_date <= user_date <= end_date):
            user_date = None
    except ValueError:
        continue

x_index = (user_date - base_date).days
act_data = machine.filtered_data
x = act_data.values[x_index - 15 : x_index, ]
x = np.array([x])

model = tf.keras.models.load_model(MODEL_PATH)
pred = model.predict(x)

pred = np.array([
    machine.scaler.inverse_transform(row)
    for row in pred
])[0]

print(pred)
print(x)

for (c, key) in enumerate(['open', 'close', 'high', 'low']):
    pred_df = pd.DataFrame(
        {f'Pred': [
            item[c] for item in pred
        ]},
        index=pd.date_range(
            user_date,
            user_date + timedelta(days=len(pred) - 1)
        )
    )
    act_df = pd.DataFrame(
        {f'Act': [
            item[c] for item in act_data.values
        ]},
        index=pd.date_range(base_date, end_date)
    )
    pd.concat([act_df, pred_df], axis=1).iplot(
        asFigure=False,
        kind='scatter',
        xTitle='Date',
        yTitle='Price',
        title=f'Price Prediction ({key})'
    )

2017-08-17 2019-04-23
What date you want to see? (min: 2017-08-17 max: 2019-04-23) 2019-01-01
[[ 9964.883  15186.295  11664.53   14365.166 ]
 [ 9913.614  17479.11   13012.741  15500.335 ]
 [ 9699.832  18211.955  13751.784  15563.566 ]
 [ 9591.58   18527.146  14124.407  15467.398 ]
 [ 9510.855  18786.963  14463.528  15458.885 ]
 [ 9504.972  18910.922  14651.663  15358.873 ]
 [ 9508.916  18998.55   14761.486  15327.917 ]
 [ 9523.588  19061.256  14836.232  15309.982 ]
 [ 9526.236  19131.26   14885.253  15340.3545]
 [ 9570.396  19152.246  14939.061  15297.177 ]
 [ 9585.908  19200.11   14967.243  15346.939 ]
 [ 9627.93   19216.158  15005.723  15295.217 ]
 [ 9645.958  19261.72   15031.618  15340.041 ]
 [ 9691.294  19264.934  15051.706  15296.734 ]
 [ 9733.159  19264.408  15052.529  15294.913 ]] [[[3229.22 3509.08 3585.   3216.  ]
  [3509.03 3652.98 3678.59 3424.47]
  [3653.   3662.22 3912.73 3622.55]
  [3660.01 4049.62 4120.   3635.  ]
  [4051.86 3838.66 4139.99 3764.  ]
  [3840.25 3948.91 3

# BTC Prediction With GRU

In [1]:
import csv
import numpy as np
from datetime import datetime, timedelta
from gru import Machine
import pandas as pd
import tensorflow as tf
from plotly.offline import init_notebook_mode
import cufflinks as cf

MODEL_PATH = "data/gru.hd5"
CSV_PATH = "data/binance-BTCUSD.csv"
TRAIN_TEST_RATE = 0.85

cf.go_offline()
init_notebook_mode(connected=True)

with open(CSV_PATH) as fin:
    reader = [dict(item) for item in csv.DictReader(fin)]
machine = Machine(reader)
(_, _, X_test, Y_test) = machine.split_data(TRAIN_TEST_RATE)

times = [int(item['openTime']) for item in reader]
times = times[int(len(times) * TRAIN_TEST_RATE) + 4:]
times = [datetime.utcfromtimestamp(t / 1000.0) for t in times]

model = tf.keras.models.load_model(MODEL_PATH)
pred = model.predict(X_test)

pred = machine.scaler.inverse_transform(pred)
Y_test = np.delete(Y_test, len(Y_test) - 1, 0)
Y_test = machine.scaler.inverse_transform(Y_test)

for (c, key) in enumerate(['open', 'close', 'high', 'low']):
    pred_df = pd.DataFrame({
        'Predictions': [item[c] for item in pred],
    }, index=times)
    act_df = pd.DataFrame({
        'Actual': [item[c] for item in Y_test]
    }, index=times[:-1])
    pd.concat([pred_df, act_df], axis=1).iplot(
        asFigure=False,
        kind='scatter',
        xTitle='Date',
        yTitle='Price',
        title=f'Price Prediction ({key})'
    )

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


KeyboardInterrupt: 

In [4]:
a = None
while not a:
    try:
        a = int(input("test"))
    except ValueError:
        continue
print(a)

testa
testa
test10
10
